In [146]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import time
from parse_str import *
import requests
import json
from datetime import datetime



In [27]:
def get_etf_attr1(symbol: str):
    time.sleep(1)

    date = pd.Timestamp.now().strftime("%Y%m%d")
    symbol = symbol.upper()
    aum = None
    expense_ratio = None
    shares_out = None
    dividend_ttm = None
    dividend_yield = None
    holdings = None

    url = Request(f"https://stockanalysis.com/etf/{symbol}/", headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(url)
    bs_obj = bs(html, "html.parser")
    trs = bs_obj.find_all('tr')

    for tr in (trs):
        tds = tr.find_all('td')
        if len(tds) > 0 :
            if tds[0].get_text() == "Assets":
                aum = tr.find_all('td')[1].get_text().replace("$", "")

            elif tds[0].get_text() == "Expense Ratio":
                expense_ratio = tr.find_all('td')[1].get_text()

            elif tds[0].get_text() == "Shares Out":
                shares_out = tr.find_all('td')[1].get_text()

            elif tds[0].get_text() == "Dividend (ttm)":
                dividend_ttm = tr.find_all('td')[1].get_text().replace("$", "")

            elif tds[0].get_text() == "Dividend Yield":
                dividend_yield = tr.find_all('td')[1].get_text()

            elif tds[0].get_text() == "Holdings":
                holdings = tr.find_all('td')[1].get_text()

    data = {
        'date': date,
        'symbol': symbol, 
        'aum': aum, 
        'expense_ratio': expense_ratio,
        'shares_out': shares_out,
        'dividend_ttm': dividend_ttm,
        'dividend_yield': dividend_yield,
        'holdings': holdings
    }
    df = pd.DataFrame([data])
    df['aum'] = df['aum'].apply(str_to_int)
    df['expense_ratio'] = df['expense_ratio'].apply(percentage_to_float)
    df['shares_out'] = df['shares_out'].apply(str_to_int)    
    df['dividend_yield'] = df['dividend_yield'].apply(percentage_to_float)
    
    return df

In [28]:
symbol = "spy"
df = get_etf_attr1(symbol)
df

,date,symbol,aum,expense_ratio,shares_out,dividend_ttm,dividend_yield,holdings
0,20240608,SPY,535010000000,0.0009,913830000,6.72,0.0126,504


In [258]:
# API URL
symbol = 'spy'
url = f"https://api.stockanalysis.com/api/symbol/e/{symbol}/overview"

# API 요청 보내기
response = requests.get(url)
data = response.json()

# ETF의 주요 정보 추출 및 출력
etf_data = data.get("data", {})

In [95]:
etf_data.keys()

dict_keys(['type', 'aum', 'nav', 'expenseRatio', 'peRatio', 'sharesOut', 'dps', 'dividendYield', 'payoutRatio', 'exDivDate', 'ch1y', 'beta', 'holdings', 'inception', 'chart', 'description', 'infoTable', 'etf_website', 'provider_page', 'holdingsTable', 'dividendTable', 'news', 'config'])